<a href="https://colab.research.google.com/github/mankicom/DEV_GDPS_TEMP_LSTM/blob/master/VAR_SEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RandomForest 및 상관관계 분석을 통한 변수선택**
단기 풍속 편차보정모델 개발 예제


### **모듈로드**


*   기본모듈: numpy, matplotlib, os, sys, time, joblib
*   변수선택 관련 모듈: RandomForestRegressor, SelectFromModel
*   사용자정의 함수: 중요도 산출 결과 bar&hline plot 함수, 상관관계 분석, 변수중요도 산출




In [ ]:
#----------------------------------------------------------------------------
# .. module load
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import time
import joblib
#from BorutaShap import BorutaShap
#from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

#.. local
sys.path.insert(0, '../inc')
from plot_importance import plot_importance_barh, plot_importance_hline
from cr_matrix import plot_correlation_matrix
from get_importance import get_feature_importances


### **랜덤포레스트 회귀 입력 및 정답자료 로드**


*   input=65종 변수 예측자료[예측시각*표본개수,변수], target=관측기온[예측시각*표본개수,변수]
*   65종 변수에 대한 변수이름=header
*   결측제거





In [ ]:
#----------------------------------------------------------------------------
# .. Load data
input_dir = "../dat/"
print_dir = "../out/"
data_per = "2016050100-2021043000-24-1605-2104"
#data_per = "sample"
#data_per = "test1stn"
input_x = "test_nwp_" + data_per
input_y = "test_obs_" + data_per

nwp = np.load(input_dir + input_x + ".npz")['value']
obs = np.load(input_dir + input_y + ".npz")['value']
header = np.load(input_dir + input_x + ".npz")['value_name']
header = header.tolist()

print (np.shape(nwp), np.shape(obs), header, type(header)) # variable, day, forecast_time
X = nwp
y = obs


print (np.shape(X), np.shape(y)) # forecast_time * day, variable

# .. remove missing
missing_idx = np.where( X==-999. )
X = np.delete( X, missing_idx[0], axis=0 )
y = np.delete( y, missing_idx[0], axis=0 )

print ( "after drop missing" )
print (np.shape(X), np.shape(y)) # forecast_time * day, variable

### **랜덤포레스트 설정 및 변수 중요도 산출**


*   랜덤포레스트 회귀모영 초모수 설정
*   fitting
*   중요도 산출
*   중요도 sorting 및 변수이름과 매치
*   학습된 랜덤포레스트 회귀모델 저장
*   오름차순으로 정렬된 변수중요도 그림 생산





In [ ]:
#----------------------------------------------------------------------------
# .. model run

# .. set model & fit
print("RandomForest Regressor ing..")
forest = RandomForestRegressor(n_jobs=-1, criterion='mse', max_depth=30,  ## max_depth=5
                           min_samples_split=3, max_features=0.33, random_state=1)
#forest = RandomForestRegressor(n_jobs=-1, criterion='mse', random_state=0)
#forest = RandomForestRegressor(n_estimators= 2, n_jobs=-1, random_state=0)
forest.fit(X, y)


# .. get feature importance
print( "Features sorted by their score:")
importances = forest.feature_importances_

# .. sort importance
indices = np.argsort(importances)[::-1]
sorted_importances= []
sorted_feature_name= []
print("Feature ranking:")
feature_name = header
for f in range(X.shape[1]-1,-1,-1):
    #print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]) )
    print("%d. feature %d %s (%f)" % (f + 1, indices[f], feature_name[indices[f]], importances[indices[f]]) )
    sorted_importances.append(importances[indices[f]])
    sorted_feature_name.append(feature_name[indices[f]])

# .. save RF result
save_name = print_dir + "rf_hline_" + data_per
print(save_name)

plot_importance_hline(sorted_feature_name, sorted_importances, save_name)
joblib.dump(forest, "../daba/" + "rf_regressor_" + data_per)

### **변수중요도 산출 결과 기반 변수선택**


*   평균 중요도 이상값 선택
*   해당하는 변수의 index 값 획득
*   해당변수의 중요도와 이름 획득

In [ ]:
#----------------------------------------------------------------------------
# .. feature select over mean

# .. select variable
sfm = SelectFromModel(estimator=forest, prefit=True, threshold='mean', max_features=None)
sel_idx=sfm.get_support()
print("sel idx: ", sel_idx)
filter_X = X[:,sel_idx]
filter_H = list(np.array(header)[sel_idx])
print("after select variable: ", filter_X.shape)

### **평균 중요도 이상 변수들의 상관관계 plot**

In [ ]:
#----------------------------------------------------------------------------
# .. correlation matrix

save_name = print_dir + "cr_matrix_" + data_per
plot_correlation_matrix(filter_X, filter_H, save_name)